In [1]:
#!pip install ultralytics

# Dataset : COCO format -> YOLO format

https://docs.ultralytics.com/datasets/detect/#ultralytics-yolo-format

In [7]:
import os
import json
import shutil

def copy_images(coco_json_path, src_image_dir, dest_image_dir):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    images = coco_data['images']

    for image_info in images:
        file_name = image_info['file_name']
        src_file_path = os.path.join(src_image_dir, file_name)
        dest_file_path = os.path.join(dest_image_dir, os.path.basename(file_name))

        os.makedirs(os.path.dirname(dest_file_path), exist_ok=True)

        if os.path.exists(src_file_path):
            shutil.copy(src_file_path, dest_file_path)
        else:
            print(f"File {src_file_path} does not exist.")

# Paths
train_coco_json = '/data/ephemeral/home/data/dataset/train_0_5.json'
val_coco_json = '/data/ephemeral/home/data/dataset/valid_0_5.json'

src_image_dir = '/data/ephemeral/home/data/dataset'
train_dest_image_dir = '/data/ephemeral/home/data/images/train'
val_dest_image_dir = '/data/ephemeral/home/data/images/valid'

# Copy images
copy_images(train_coco_json, src_image_dir, train_dest_image_dir)
copy_images(val_coco_json, src_image_dir, val_dest_image_dir)

In [8]:
# labels 만들기
import os
import json

def convert_coco_to_yolo(coco_json_path, output_dir, image_dir):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    images = {img['id']: img for img in coco_data['images']}
    annotations = coco_data['annotations']
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}

    os.makedirs(output_dir, exist_ok=True)

    for ann in annotations:
        image_id = ann['image_id']
        image_info = images[image_id]
        file_name = image_info['file_name']
        width = image_info['width']
        height = image_info['height']

        bbox = ann['bbox']
        x_center = (bbox[0] + bbox[2] / 2) / width
        y_center = (bbox[1] + bbox[3] / 2) / height
        w = bbox[2] / width
        h = bbox[3] / height

        category_id = ann['category_id']
        yolo_annotation = f"{category_id} {x_center} {y_center} {w} {h}\n"

        label_file_name = os.path.splitext(file_name)[0].replace('train/', '') + '.txt'
        label_file_path = os.path.join(output_dir, label_file_name)

        with open(label_file_path, 'a') as label_file:
            label_file.write(yolo_annotation)

# Paths
train_coco_json = '/data/ephemeral/home/data/images/train_0_5.json'
val_coco_json = '/data/ephemeral/home/data/images/valid_0_5.json'

train_image_dir = '/data/ephemeral/home/data/images/train'
val_image_dir = '/data/ephemeral/home/data/images/train'

train_output_dir = '/data/ephemeral/home/data/labels/train'
val_output_dir = '/data/ephemeral/home/data/labels/val'

# Ensure output directories exist
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(val_output_dir, exist_ok=True)

# Convert COCO to YOLO
convert_coco_to_yolo(train_coco_json, train_output_dir, train_image_dir)
convert_coco_to_yolo(val_coco_json, val_output_dir, val_image_dir)

<class_id> <x_center> <y_center> <width> <height>

In [7]:
# Paths
test_coco_json = '/data/ephemeral/home/data/images/test.json'

test_image_dir = '/data/ephemeral/home/data/images/test'

test_output_dir = '/data/ephemeral/home/data/labels/test'

# Ensure output directories exist
os.makedirs(test_output_dir, exist_ok=True)

# Convert COCO to YOLO
convert_coco_to_yolo(test_coco_json, test_output_dir, test_image_dir)